# TP3 - Optimizing with Constraint Programming - The Golomb ruler example

### Duc Hau NGUYEN - Adrian MEGA

## Golomb ruler

Your goal is to place $N$ marks on a ruler, such that no two marks are at the same distance and the total length of the ruler (the position of the last mark) is minimized. 

<div class="row" style="margin-top: 10px">
    <img src="display/images/Golomb_Ruler-4.svg" style="display: block; margin: auto; width: 400px;" />
    <p style="margin: auto; margin-top: 10px; text-align: center;">Golomb ruler of order 4 and length 6. This ruler is both optimal and perfect.</p>
</div>

Golomb rule can be used in information theory to design error correcting codes or in telecommunications to avoid interferences during radio communications.

**Exercice**: Create differents models for the decision version of the Golomb ruler and implement it using `docplex`. For $N$ marks, we can suppose that the maximum length of the rules is $2 N$.

In [19]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
from config import setup
setup()

In [88]:
def create_golomb_ruler_1(N):
    '''
    Model all_diff
    @param N: number of marks
    '''
    mdl = CpoModel(name='Golomb ruler')
    
    # create model
    marks = mdl.integer_var_list(N, 0, 2 * N, "marks")
    
    # constraint
    mdl.add(all_diff(marks))
    mdl.add(all_diff( abs(marks[i] - marks[j]) for i in range(N) for j in range(i+1,N) )  )
    mdl.add(all_diff( abs(marks[i] - marks[j]) for i in range(N) for j in range(i)     )  )
            
    return mdl

In [140]:
# Param
N = 4
mdl = create_golomb_ruler_1(N)
sol = mdl.solve()
print(sol)

-------------------------------------------------------------------------------
Model constraints: 24, variables: integer: 4, interval: 0, sequence: 0
Solve status: Feasible, Fail status: SearchHasNotFailed
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------

marks_0: 0
marks_1: 4
marks_2: 3
marks_3: 1



In [142]:
def create_golomb_ruler_2(N):
    '''
    Model all_diff
    @param N: number of marks
    '''
    mdl = CpoModel(name='Golomb ruler')
    
    # create model
    marks = mdl.integer_var_list(N, 0, 2 * N, "marks")
    
    # constraint
    mdl.add(all_diff(marks))
    mdl.add(marks[i] > marks[i-1] for i in range(1,N))
    mdl.add(all_diff( abs(marks[i] - marks[j]) for i in range(N) for j in range(i+1,N) )  )
    mdl.add(all_diff( abs(marks[i] - marks[j]) for i in range(N) for j in range(i)     )  )
            
    return mdl

In [147]:
# Param
N = 4
mdl = create_golomb_ruler_2(N)
sol = mdl.solve()
print(sol)

-------------------------------------------------------------------------------
Model constraints: 6, variables: integer: 4, interval: 0, sequence: 0
Solve status: Feasible, Fail status: SearchHasNotFailed
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------

marks_0: 0
marks_1: 1
marks_2: 3
marks_3: 7



In [138]:
def create_golomb_ruler_3(N):
    '''
    Model single
    @param N: number of marks
    '''
    mdl = CpoModel(name='Golomb ruler')
    
    # create model
    marks = mdl.integer_var_list(N, 0, 2 * N, "marks")
    
    # constraint
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if(i != j and j != k and k != i):
                    dist_ik = abs(marks[i] - marks[k])
                    dist_jk = abs(marks[j] - marks[k])
                    mdl.add(dist_ik != dist_jk)
                
    return mdl

**Exercice**: Solve the model for different values of N.

In [93]:
for N in range(8):
    mdl = create_golomb_ruler_1(N)
    sol = mdl.solve()
    print(sol)

-------------------------------------------------------------------------------
Model constraints: 3, variables: integer: 0, interval: 0, sequence: 0
Solve status: Feasible, Fail status: SearchHasNotFailed
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------


-------------------------------------------------------------------------------
Model constraints: 3, variables: integer: 1, interval: 0, sequence: 0
Solve status: Feasible, Fail status: SearchHasNotFailed
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------

marks_0: 0

-------------------------------------------------------------------------------
Model constraints: 3, variables: integer: 2, interval: 0, sequence: 0
Solve status: Feasible, Fail status: SearchHasNotFailed
Search status: SearchCompleted,

**Exercice**: Add the objective to the different models, and run them for different values of N. Evaluate the different models with different heuristics. 

In [173]:
def create_unbound_ruler_1(N):
    '''
    Model all_diff
    @param N: number of marks
    '''
    mdl = CpoModel(name='Golomb ruler')
    
    # create model
    marks = mdl.integer_var_list(N, 0, N**2, "marks")
    
    # constraint
    mdl.add(all_diff(marks))
    mdl.add(marks[i] > marks[i-1] for i in range(1,N))
    mdl.add(all_diff( abs(marks[i] - marks[j]) for i in range(N) for j in range(i+1,N) )  )
    mdl.add(all_diff( abs(marks[i] - marks[j]) for i in range(N) for j in range(i)     )  )
    
    mdl.add_search_phase(search_phase(marks,
                                          varchooser = select_random_var(),
                                          valuechooser = select_smallest(value_index(range(N)))))

            
    mdl.add(minimize(marks[N - 1]))
    
    return mdl

In [174]:
N = 3
mdl = create_unbound_ruler_1(N)
sol = mdl.solve(Presolve='On')
print(sol)

-------------------------------------------------------------------------------
Model constraints: 5, variables: integer: 3, interval: 0, sequence: 0
Solve status: Optimal, Fail status: SearchHasFailedNormally
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------
Objective values: (3,), bounds: (3,), gaps: (0.0,)
marks_0: 0
marks_1: 1
marks_2: 3



In [137]:
for N in range(3,8) :
    mdl = create_unbound_ruler_1(N)
    sol = mdl.solve(Presolve='On')
    print(sol)

-------------------------------------------------------------------------------
Model constraints: 5, variables: integer: 3, interval: 0, sequence: 0
Solve status: Optimal, Fail status: SearchHasFailedNormally
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------
Objective values: (3,), bounds: (3,), gaps: (0.0,)
marks_0: 0
marks_1: 1
marks_2: 3

-------------------------------------------------------------------------------
Model constraints: 6, variables: integer: 4, interval: 0, sequence: 0
Solve status: Optimal, Fail status: SearchHasFailedNormally
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.0 sec
-------------------------------------------------------------------------------
Objective values: (6,), bounds: (6,), gaps: (0.0,)
marks_0: 0
marks_1: 1
marks_2: 4
marks_3: 6

---------------------------------------------------------------